# EDA example
This notebook is an example of how to use our ConceptDriftsFinder tool as part of the EDA process.

## Step 1 - Technical initialization
We will start with a few technical dataset loading steps and notebook configuration.

The dataset we will use in this example is the housing dataset. See more about the dataset in the README.md file, the accompayning pdf or the `datasets_config.py` file.

### Install necessary requirements

In [ ]:
%pip install -r ../requirements.txt

### Change working directory and add jupyter reload

In [ ]:
# Change working directory to root
import os
if os.getcwd().endswith("notebooks"):
    %cd ..
    print(os.getcwd())

# Automatically reload changes in code
%load_ext autoreload
%autoreload 2

### Imports, logging and pandas configuration

In [ ]:
import logging
from typing import List
import pandas as pd
from association_finder.concept_drifts_finder import ConceptDriftsFinder, convert_df_to_transactions
from association_finder.models import Transaction, ConceptDriftResult
from association_finder.concept_engineering import ConceptEngineering
from association_finder.datasets_config import datasets_config
from sklearn.model_selection import train_test_split
from association_finder.preprocessing import preprocess_dataset, split_X_y
from association_finder.one_vs_rest_classifier import OneVsRestClassifier, label_to_concept_transform_wrapper
from typing import Dict, Tuple, Optional
from dataclasses import dataclass
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Logs config
logging.basicConfig(level=logging.DEBUG)

# Pandas config
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)

### Read, split and preprocess data

In [ ]:
np.random.seed(0)

# Use housing dataset for the example
dataset_config = datasets_config["housing"]

# Read file
df = pd.read_csv(dataset_config["train_dataset_path"], index_col=dataset_config['index_col'])
target_column = dataset_config["target_column"]

# Drop rows with NaN values in the target column.
df.drop(df[df[target_column].isna()].index,inplace=True)

# Split
df_train, df_val = train_test_split(df, test_size=0.3)

# Preprocess    
df_train_prep, train_params = preprocess_dataset(df_train)

# Focusing on prominent columns:
good_columns = [column for column in dataset_config["good_columns"] if column not in train_params.dropped_columns]
one_hot_columns = [column for column in dataset_config["one_hot_columns"] if column not in train_params.dropped_columns]

## Step 2 - Using ConceptsDriftFinder
We are now ready to start using ConceptsDriftFinder.

Let's start by analyzing potential concept drifts. An immediate suspect would be the `YearBuilt` column.

In [ ]:
transactions = convert_df_to_transactions(df_train_prep)
ConceptDriftsFinder().find_concept_drifts(transactions, concept_column="YearBuilt", target_column=target_column)